# Day 13: Mine Cart Madness
[link](https://adventofcode.com/2018/day/13)

## Parsing inputs

In [1]:
test_in = """/->-\\        
|   |  /----\\
| /-+--+-\\  |
| | |  | v  |
\-+-/  \\-+--/
  \------/   """.split('\n')

print('\n'.join(test_in))
print(set(len(s) for s in test_in))

/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   
{13}


In [2]:
with open('input.txt') as file:
  puzzle_in = [line.strip('\n') for line in file]

print(set(len(s) for s in puzzle_in))

{150}


## Part 1

Find `X,Y` coordinates of the first crash.

In [3]:
# Direction after a curve
curve_turns = {'>/':'^','>\\':'v','</':'v','<\\':'^','v/':'<','v\\':'>','^/':'>','^\\':'<'}

# Direction after an intersection depending on whether the cart wants to go left 'l' or right 'r'
intersection_turns = {'>l':'^','>r':'v','<l':'v','<r':'^','vr':'<','vl':'>','^r':'>','^l':'<'}

# Position increment depending on the cart's direction
position_increment = {'>': (1,0),'<': (-1,0),'^': (0,-1),'v': (0,1)}


# Generator of turn directions, which infinitely produces 'l', 's', 'r', 's', ...
def turn_direction_generator():
  while True:
    for turn in 'lsrs':
      yield turn


class Cart:
  def __init__(self, direction):
    self.direction = direction
    self.turn_direction_generator = turn_direction_generator()


  def update_direction(self, track):
    if track == '+':  # Intersection
      turn_direction = next(self.turn_direction_generator)
      if turn_direction in 'lr':  # Turn either left or right
        self.direction = intersection_turns[self.direction + turn_direction]
      else:  # Go straight - no need to change direction
        assert turn_direction == 's'
    elif track in '\/':  # Curve
      self.direction = curve_turns[self.direction + track]
    else:  # Straight
      assert track in '-|'


  def next_position(self, x, y):
    increment = position_increment[self.direction]
    return x+increment[0], y+increment[1]


  def __repr__(self):
    return self.direction

In [4]:
class Mine:
  def __init__(self, input_str):
    self.carts = {}
    self.tracks = []

    for y,line in enumerate(input_str):
      track_line = []
      for x,char in enumerate(line):
        if char in '<v>^':
          assert (y,x) not in self.carts
          self.carts[y,x] = Cart(char)
          track_line.append('|' if char in '^v' else '-')
        else:
          track_line.append(char)
      self.tracks.append(track_line)


  def __repr__(self):
    return '\n'.join(''.join(char if (y,x) not in self.carts else self.carts[y,x].direction for x,char in enumerate(line)) for y,line in enumerate(self.tracks))


  def tick(self):
    for y,x in sorted(self.carts.keys()):
      cart = self.carts.pop((y,x))
      new_x,new_y = cart.next_position(x,y)

      new_track = self.tracks[new_y][new_x]
      cart.update_direction(new_track)

      if (new_y,new_x) in self.carts:  # Collision
        self.carts[new_y,new_x].direction = 'X'
        raise Exception(f'Collision at ({new_x},{new_y})')

      self.carts[new_y,new_x] = cart

In [5]:
def print_mine(mine,i=0):
  print(f'{i:02}\n{mine}')

def evolve_mine(input_str,ticks_number=2000,debug=True):
  mine = Mine(input_str)
  assert str(mine)=='\n'.join(input_str)
  if debug:
    print_mine(mine)

  for i in range(1,ticks_number):
    try:
      mine.tick()
      if debug:
        print_mine(mine,i)
    except Exception as exception:
      print(f'\n{exception}')
      print_mine(mine,i)
      break


evolve_mine(test_in)

00
/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   
01
/-->\        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \->--/
  \------/   
02
/---v        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+>-/
  \------/   
03
/---\        
|   v  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+->/
  \------/   
04
/---\        
|   |  /----\
| /->--+-\  |
| | |  | |  |
\-+-/  \-+--^
  \------/   
05
/---\        
|   |  /----\
| /-+>-+-\  |
| | |  | |  ^
\-+-/  \-+--/
  \------/   
06
/---\        
|   |  /----\
| /-+->+-\  ^
| | |  | |  |
\-+-/  \-+--/
  \------/   
07
/---\        
|   |  /----<
| /-+-->-\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   
08
/---\        
|   |  /---<\
| /-+--+>\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   
09
/---\        
|   |  /--<-\
| /-+--+-v  |
| | |  | |  |
\-+-/  \-+--/
  \------/   
10
/---\        
|   |  /-<--\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   
11
/---\        
|   |  /<---\
| /-+--+-\  

In [6]:
evolve_mine(puzzle_in,debug=False)


Collision at (35,50)
80
                                          /-----------------------------------------------------------------------------------------------------\     
                         /----------------+---------------------------------------------------------------------------------\                   |     
  /----------------------+----------------+--------------------------------------------------------------\                  |                   |     
  |  /----------\        |             /--+--------------<---------------------------------\             |                  |                   |     
  |  |          |/-------+---------\   |  |           /------------------------------------+-------------+\                 |                   |     
  |  |          ||       |       /-+---+--+-----------+------------------------------------+-------------++-----------\     |                   |     
  |/-+----------++-------+-\     | |   |  |           |              

In [7]:
puzzle_mine = Mine(puzzle_in)
assert str(puzzle_mine)=='\n'.join(puzzle_in)  # Check if the initial tracks + carts representation is the same as the puzzle input

for i in range(2000):
  puzzle_mine.tick()

Exception: Collision at (35,50)

**Incorrect answers:**
- `35,50` is not the right answer
- `36,87` is not the right answer

In [ ]:
with open('test_2.txt') as file:
  test_in_2 =[line.strip('\n') for line in file]

print(set(len(s) for s in test_in_2))
print('\n'.join(test_in_2))

In [ ]:
test_mine_2 = Mine(test_in_2)
assert str(test_mine_2)=='\n'.join(test_in_2)
print(f'00 {test_mine_2.carts}\n{test_mine_2}\n')

for i in range(100):
  test_mine_2.tick()
  print(f'=== {i+1:02} ===\n{test_mine_2}')